In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY
from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

from loguru import logger
import matplotlib.pyplot as plt
logger.enable("gptchem")

In [3]:
import decorator, time

def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get('timeout', 0.0) # seconds
    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try: return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None: time.sleep(timeout)
    return tryIt

In [4]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
        "matbench_expt_gap",
        "matbench_steels",
    ],
)

2023-01-28 21:22:48 INFO     Initialized benchmark 'matbench_v0.1' with 2 tasks: 
['matbench_expt_gap', 'matbench_steels']


In [7]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(task.metadata['target'], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False))
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)
    print(predictions)
    # Record your data!
    task.record(fold, predictions)

In [8]:
task.scores

NameError: name 'task' is not defined

In [9]:
for task in mb.tasks:
    task.load()
    
    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
           print(f"Skipping fold {fold_ind} of {task.dataset_name}")
           continue
        train_test_fold(task, fold)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-28 21:39:00 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 151Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230128_213900/train.jsonl: file-4yNHcICnWA90BLajzW5YOg0W


2023-01-28 21:39:05.687 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1674938345,
  "events": [
    {
      "created_at": 1674938345,
      "level": "info",
      "message": "Created fine-tune: ft-8gItVsA92myX99OBzNCPLg24",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-8gItVsA92myX99OBzNCPLg24",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1674938342,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230128_213900/train.jsonl",
      "id": "file-4yNHcICnWA90BLajzW5YOg0W",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      

In [ ]:
mb.to_file("gpt_bench.json.gz")